<a href="https://colab.research.google.com/github/gDan15/AI_colab/blob/master/Model_elaboration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Check and install tensorflow

In [0]:
# To determine which version you're using:
!pip show tensorflow

Name: tensorflow
Version: 1.14.0rc0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, tensorboard, keras-preprocessing, gast, absl-py, astor, wheel, wrapt, google-pasta, tf-estimator-nightly, numpy, protobuf, grpcio, termcolor, keras-applications
Required-by: stable-baselines, magenta, fancyimpute


In [0]:
# For the current version: 
# !pip install --upgrade tensorflow

# For installing previous version
 !pip install tensorflow==1.14.0rc0

# For the latest nightly build:
# !pip install tf-nightly

In [0]:
!pip install Pillow

## Load files

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
pwd

'/content'

In [0]:
cd /content/gdrive/My\ Drive/Colab Notebooks/

/content/gdrive/My Drive/Colab Notebooks


In [0]:
ls

 AI.ipynb       drive.ipynb    IMG_heavy/   IMG_old/             test.ipynb
 checkpoints/   driving_log/   IMG_light/  'Model elaboration'


## Packages

In [0]:
import csv, random, numpy as np
import tensorflow as tf
import PIL
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import img_to_array, load_img, random_shift
from tensorflow.keras.models import load_model, Sequential
from PIL import Image
from tensorflow.keras.callbacks import TensorBoard

# from tensorflow.keras.models import load_model, Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten
# from tensorflow.keras.layers.convolutional import Convolution2D, MaxPooling2D
# from tensorflow.keras.preprocessing.image import img_to_array, load_img, flip_axis, random_shift

## Model

In [0]:
def model(load, shape, checkpoint=None):
  if load and checkpoint: return load_model(checkpoint)
  conv_layers, dense_layers = [32, 32, 64, 120], [1024, 512]
  model = tf.keras.models.Sequential()
  model.add(layers.Conv2D(32, 3, strides=1, activation='elu', input_shape=shape))
  model.add(layers.MaxPooling2D())
  for cl in conv_layers:
      model.add(layers.Conv2D(filters=cl, kernel_size=3, strides=1, activation='elu'))
      model.add(layers.MaxPooling2D())
  model.add(layers.Flatten())
  for dl in dense_layers:
      model.add(layers.Dense(dl, activation='elu'))
      model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='linear'))
#   tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
  model.compile(loss='mse', optimizer="adam", metrics=['accuracy'])
  return model

### Testing model

In [0]:
net = model(load=False, shape=(100, 100, 3))
net.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_132 (Conv2D)          (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_124 (MaxPoolin (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_133 (Conv2D)          (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_125 (MaxPoolin (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_134 (Conv2D)          (None, 21, 21, 32)        9248      
_________________________________________________________________
max_pooling2d_126 (MaxPoolin (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_135 (Conv2D)          (None, 8, 8, 64)        

## Process data

In [0]:
def get_X_y(data_file, img):
    """Read the log file and turn it into X/y pairs. Add an offset to left images, remove from right images."""
    X, y = [], []
    # 0.4
    steering_offset = 0
    with open(data_file) as fin:
        for center_img, left_img, right_img, steering_angle, _, _, speed in csv.reader(fin):
            if float(speed) < 20: continue  # throw away low-speed samples
#            if abs(float(steering_angle)) < 0.05: continue
            left_img = left_img.strip()
            left_img = "/content/gdrive/My Drive/Colab Notebooks/" + img + "/" + left_img.split("/")[-1]
            right_img = left_img.strip()
            right_img = "/content/gdrive/My Drive/Colab Notebooks/" + img + "/" + right_img.split("/")[-1]
            center_img = center_img.strip()
            center_img = "/content/gdrive/My Drive/Colab Notebooks/" + img + "/" + center_img.split("/")[-1]
#             X += [left_img, right_img, center_img]
            X += [center_img]
            y += [float(steering_angle) + steering_offset, float(steering_angle) - steering_offset]
    return X, y

In [0]:
# X, y = get_X_y('/content/gdrive/My Drive/Colab Notebooks/driving_log/driving_log_16_06.csv')
# print(X)
# print(y)

## Process image

In [0]:
def process_image(path, steering_angle, augment, shape=(100,100)):
    """Process and augment an image."""
    image = Image.open(path)
    image = resize_image(image)
#     image = load_img(path, target_size=shape)
    
    if augment and random.random() < 0.5:
         image = random_darken(image)  # before numpy'd

    image = img_to_array(image)
        
    if augment:
        image = random_shift(image, 0, 0.2, 0, 1, 2)  # only vertical
        if random.random() < 0.5:
#            image = apply_transform(image, "flip_horizontal")
            steering_angle = -steering_angle

    image = (image / 255. - .5).astype(np.float32)
    return image, steering_angle

In [0]:
def random_darken(image):
    """Given an image (from Image.open), randomly darken a part of it."""
    #grace_hopper = tf.keras.utils.get_file(image_path)
    
    w, h = image.size

    # Make a random box.
    x1, y1 = random.randint(0, w), random.randint(0, h)
    x2, y2 = random.randint(x1, w), random.randint(y1, h)

    # Loop through every pixel of our box (*GASP*) and darken.
    for i in range(x1, x2):
        for j in range(y1, y2):
            new_value = tuple([int(x * 0.5) for x in image.getpixel((i, j))])
            image.putpixel((i, j), new_value)
    return image

In [0]:
def resize_image(image):
    image = image.crop((0, 60, 320, 160))
    image = image.resize((100,100), Image.ANTIALIAS)
    return image
    

### Test process image

#### Random_darken

In [0]:
path = "IMG_light/center_2019_06_16_19_11_36_418.jpg"

In [115]:
i = Image.open(path)
random_darken(i)

OSError: ignored

#### Crop and resize

In [0]:
image = Image.open(path)
resize_image(image) 

NameError: ignored

## Train model

### Generator

In [0]:
def _generator(batch_size, X, y):
    """Generate batches of training data forever."""
    print(len(X))
    while 1:
        batch_X, batch_y = [], []
        for i in range(batch_size):
            sample_index = random.randint(0, len(X) - 1)
            sa = y[sample_index]
            try:
              image, sa = process_image(X[sample_index], sa, augment=True)
              batch_X.append(image)
              batch_y.append(sa)
            except FileNotFoundError:
              print("File not found : " + X[sample_index])
              next
        yield np.array(batch_X), np.array(batch_y)


In [0]:
path_validation = "/content/gdrive/My Drive/Colab Notebooks/driving_log/driving_log_heavy.csv"
img_validation = "IMG_heavy"
test_validation = validation_set(path_validation, img_validation)
def train():
    """Load our network and our data, fit the model, save it."""
    driving_log = "driving_log_heavy.csv"
    img = "IMG_heavy"
    net = model(load=False, shape=(100, 100, 3))
#     net = model(load=True, shape=(100, 100, 3), checkpoint='checkpoints/short_light.h1')
    X, y = get_X_y('/content/gdrive/My Drive/Colab Notebooks/driving_log/' + driving_log, img)
#     net.fit_generator(_generator(100, X, y), steps_per_epoch=1, epochs=1, validation_data=test_validation)
    net.fit_generator(_generator(256, X, y), steps_per_epoch=50, epochs=160)
    net.save('checkpoints/short_heavy.h5_9')
    
    driving_log = "driving_log_light.csv"
    img = "IMG_light"
    """Load our network and our data, fit the model, save it."""
    net = model(load=False, shape=(100, 100, 3))
#     net = model(load=True, shape=(100, 100, 3), checkpoint='checkpoints/short_light.h1')
    X, y = get_X_y('/content/gdrive/My Drive/Colab Notebooks/driving_log/' + driving_log, img)
#     net.fit_generator(_generator(100, X, y), steps_per_epoch=1, epochs=1, validation_data=test_validation)
    net.fit_generator(_generator(256, X, y), steps_per_epoch=50, epochs=160)
    net.save('checkpoints/short_light.h1_4')

In [0]:
def load(checkpoint):
  return load_model(checkpoint)

In [0]:
cd /content/gdrive/My\ Drive/Colab Notebooks/

/content/gdrive/My Drive/Colab Notebooks


### Start

In [0]:
if __name__ == '__main__':
  train()
#     net = .load('checkpoints/short_doesnt_work.h5')
#     print(net)

## Validation set

In [0]:
def validation_set(data_file, img):
  i = 0
  X, y, test_x, test_y = [], [], [], []
  with open(data_file) as fin:
    for center_img, _, _, steering_angle, _, _, speed in csv.reader(fin):
      if float(speed) < 20: continue  # throw away low-speed samples
      center_img = center_img.strip()
      center_img = "/content/gdrive/My Drive/Colab Notebooks/" + img + "/" + center_img.split("/")[-1]
      X += [center_img]
      y += [float(steering_angle)]
      i+=1
      if i == 20: break
  for source, target in zip(X, y):
    try:
      image, sa = process_image(source, target, augment=True)
      test_x.append(image)
      test_y.append(sa)
    except FileNotFoundError:
      print("File not found : " + X[sample_index])
      next
  return (test_x, test_y)

In [0]:
path = "/content/gdrive/My Drive/Colab Notebooks/driving_log/driving_log_light.csv"
img = "IMG_light"
validation_set(path, img)

## Validate model

In [0]:
def test_model(model, data_file, img):
  X, y = [], []
  with open(data_file) as fin:
        for center_img, left_img, right_img, steering_angle, _, _, speed in csv.reader(fin):
            if float(speed) < 20: continue  # throw away low-speed samples
            center_img = center_img.strip()
            center_img = "/content/gdrive/My Drive/Colab Notebooks/" + img + "/" + center_img.split("/")[-1]
#             X += [left_img, right_img, center_img]
            image = Image.open(center_img)
            image = resize_image(image)
            image = img_to_array(image)
            image = (image / 255. - .5).astype(np.float32)
            #image = image.reshape((-1, 100, 100, 3))
            X += [image]
            y += [float(steering_angle)]
  X = np.array(X)
  print("Source : {0}, Target values : {1}".format(len(X), len(y)))
  results = model.evaluate(X, y, batch_size=100, steps=100)
  return results

In [0]:
net = model(load=True, shape=None, checkpoint='checkpoints/short_heavy.h5')
path = "/content/gdrive/My Drive/Colab Notebooks/driving_log/driving_log_heavy.csv"
img = "IMG_heavy"
test_model(net, path, img)

Source : 4436, Target values : 4436
100/100 [==============================] - 40s 400ms/step - loss: 0.0309 - acc: 0.8043


[0.030883772298693657, 0.8043282]